## image crop code (x, y 좌표값)

In [2]:
import cv2
import numpy as np
import os
import tensorflow as tf

def parse_tfrecord(example):
    features = {
        'Raw_data_ID': tf.io.FixedLenFeature([], tf.string),
        'breed': tf.io.FixedLenFeature([], tf.string),
        'age': tf.io.FixedLenFeature([], tf.int64),
        'gender': tf.io.FixedLenFeature([], tf.string),
        'species': tf.io.FixedLenFeature([], tf.string),
        'lesions': tf.io.FixedLenFeature([], tf.string),
        'path': tf.io.FixedLenFeature([], tf.string),
        'img_path': tf.io.FixedLenFeature([], tf.string),
        'x_coords': tf.io.VarLenFeature(tf.int64),
        'y_coords': tf.io.VarLenFeature(tf.int64),
    }

    example = tf.io.parse_single_example(example, features)
    
    # 이미지 경로 수정
    image_path = tf.strings.join([example['img_path'], tf.constant('/'), example['Raw_data_ID']])

    print(image_path)

    # 좌표를 리스트로 변환
    x_coords = tf.sparse.to_dense(example['x_coords'], default_value=-1)  # 빈 좌표는 -1로 처리
    y_coords = tf.sparse.to_dense(example['y_coords'], default_value=-1)  # 빈 좌표는 -1로 처리
    
    # 좌표 튜플로 반환
    coords_pairs = (x_coords, y_coords)
    return image_path, example['lesions'], coords_pairs



# TFRecord 파일 불러오는 함수
def load_dataset(tfrecord_file):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_tfrecord)
    return dataset

# 좌표값을 이용한 image crop 코드
def create_true_mask(image_path, x_coords, y_coords, save_dir):
    # 이미지 경로를 문자열로 변환
    image_path = image_path.numpy().decode('utf-8')

    # 이미지 불러오기
    image = cv2.imread(image_path)
    image_copy = image.copy()

    # 다각형 좌표를 NumPy 배열로 변환
    pts = np.array(list(zip(x_coords.numpy(), y_coords.numpy())), np.int32)
    pts = pts.reshape((-1, 1, 2))  # 각각의 좌표 쌍을 점으로 변환
    print(pts)

    # 다각형 그리기
    cv2.polylines(image_copy, [pts], isClosed=True, color=(255, 255, 255), thickness=2)

    # 다각형 내부를 채워 마스킹
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, [pts], color=(255, 255, 255))

    # 마스크를 이용하여 원본 이미지에서 다각형 부분만 추출
    masked_image = cv2.bitwise_and(image, mask)

    # 마스크 이미지 저장
    filename = os.path.basename(image_path)
    mask_save_path = os.path.join(save_dir, f"mask_{filename}")
    cv2.imwrite(mask_save_path, masked_image)

    print(f"image crop 이미지가 생성되었습니다. 저장된 경로: {mask_save_path}")


tfrecord_file = './train_symptoms_tfrecords/train_symptoms_A1.tfrecord'
dataset = load_dataset(tfrecord_file)

for example in dataset:
    image_path, lesions, coords_pairs = example
    print('img_path:', image_path.numpy().decode('utf-8'))
    print("Lesion:", lesions.numpy().decode('utf-8'))

    x_coords, y_coords = coords_pairs
    print("X Coords:", x_coords.numpy())
    print("Y Coords:", y_coords.numpy())
    
    # 저장할 디렉토리 생성
    save_dir = './train_crop_images_A1'
    os.makedirs(save_dir, exist_ok=True)
    create_true_mask(image_path, x_coords, y_coords, save_dir)